In [336]:
%store -r data
%store -r le

In [317]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py
init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username='lyndongreenhouse', api_key='2OB7yoHDZgz8GxKxC9y4')

In [337]:
data["duration"] = pd.to_timedelta(data["deadline"]  - data['launched'])

In [338]:
success_proj = data[data["state"] == True]
suc_cat = pd.DataFrame(success_proj.main_category.value_counts())
suc_cat.columns = ["suc_main_cat"]
suc_cat["total"] = data.main_category.value_counts()
suc_cat["rate"] = suc_cat.suc_main_cat / suc_cat.total * 100
suc_cat.index = le.inverse_transform(suc_cat.index)
suc_cat.sort_values(by="rate",inplace=True)

In [339]:
gdata = [go.Bar(x = suc_cat.index, y = suc_cat.suc_main_cat, name = "Success Number"),
        go.Bar(x = suc_cat.index, y = suc_cat.total,name = "Total Number"),
        go.Scatter(x = suc_cat.index, y = suc_cat.rate,yaxis='y2', name = "Success Rate")
       ]

In [340]:
iplot(go.Figure(data = gdata, 
        layout = go.Layout(
            title='Main Catagory Successful Rate',
        yaxis=dict(
            title='Number of Projects'
        ),
        yaxis2=dict(
            title='Successful Rate',
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            overlaying='y',
            side='right'
        )
        )
     )
     )

In [346]:
success_proj = data[data["state"] == True]
suc_sub_cat = pd.DataFrame(success_proj.category.value_counts())
suc_sub_cat.columns = ["suc_cat"]
suc_sub_cat["total"] = data.category.value_counts()
suc_sub_cat["rate"] = suc_sub_cat.suc_cat / suc_sub_cat.total * 100
suc_sub_cat.index = le.inverse_transform(suc_sub_cat.index)
suc_sub_cat.sort_values(by="rate",inplace=True)

gdata = [go.Bar(x = suc_sub_cat.index, y = suc_sub_cat.suc_cat, name = "Success Number"),
        go.Bar(x = suc_sub_cat.index, y = suc_sub_cat.total,name = "Total Number"),
        go.Scatter(x = suc_sub_cat.index, y = suc_sub_cat.rate,yaxis='y2', name = "Success Rate")
       ]

iplot(go.Figure(data = gdata, 
        layout = go.Layout(
            title='Sub Catagory Successful Rate',
        yaxis=dict(
            title='Number of Projects'
        ),
        yaxis2=dict(
            title='Successful Rate',
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            overlaying='y',
            side='right'
        )
        )
     )
     )

In [347]:
data.head(2)

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,duration
ID,,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,160,43,23,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,False,0,22,0.0,0.0,1533.95,58 days 11:47:32
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,145,20,51,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,False,15,50,100.0,2421.0,30000.00,59 days 19:16:03


In [128]:
#sub -> main catagory -> country -> duration -> goal   -> state

In [348]:
gsource = []
gtarget =  []
gvalue =  []
glabel =  []

In [349]:
data["duation_cat"] = pd.cut(data.duration, bins = 1000).astype(str)
data["goal_cat"] = pd.cut(data.goal, bins = 20).astype(str)

In [350]:
subcat = list(set(le.inverse_transform(data.category)))
cat = list(set(le.inverse_transform(data.main_category)))
for i in range(len(subcat)):
    if subcat[i] in cat:
        subcat[i] = subcat[i]+"_sub"
cou = list(set(le.inverse_transform(data.country)))
dua = list(set(data.duation_cat))
goal = list(set(data.goal_cat))
total = subcat + cat+cou+dua+goal

In [351]:
#Sub -> Main
temp = sorted(list(set(zip(data.category,data.main_category))),key=lambda x: x[1])
value = [list(zip(data.category,data.main_category)).count(x) for x in temp]
gsource = [total.index(le.inverse_transform([x[0]])[0]) for x in temp]
gtarget = [total.index(le.inverse_transform([x[1]])[0]) for x in temp]
gvalue = value

# Main -> COuntry
temp = sorted(list(set(zip(data.main_category,data.cu))),key=lambda x: x[1])
value = [list(zip(data.category,data.main_category)).count(x) for x in temp]

In [362]:
data_trace = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(
        color = "black",
        width = 0.5
      ),
        label = total,
        color = ["blue"]*len(total)
    ),
    link = dict(
      source =  gsource,
      target =  gtarget,
      value =  gvalue
  )
)

layout =  go.Layout(
    title = "Test",
    font = dict(
      size = 10
    )
)

fig = go.Figure(data=[data_trace],layout = layout)
iplot(fig, validate=True)